<a href="https://colab.research.google.com/github/ashfaq1192/langchain_rag/blob/main/project_2_Langchain_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 2: Langchain RAG Project

# ***RAG***

Retrieved Augmented Generation (RAG) is a system or tool used for storing the embedded data which is used by LLM when we ask a domain specific or technical question from the LLM. Our prompt first goes to the RAG Vector Database and collects the required embedded data and then goes towards the LLM which give use the specific required output in concise form with the help of given data through RAG alongwith its own previous knowledge which was given at the time of training of LLLM. Because, LLMs are trained till specific date, so for example, if we want the LLM to give us the details or required information from the list of projects we have done which definitely LLM does not know. So,for that purpose, we have to store our all projects in RAG Vector Database and then we will ask the question from the LLM which will first go to the Vector Database and then will go towards LLM and then LLM will be such in a position that it can answer our questions efficiently.

1. RAG is used for Searching data from Vector Database. It is like a bag in which we have everthing about the specific domain and we direct the LLM to use that bag before answering so that our answer can be free from hallucinations and the LLM gives us the answer after searching from the RAG.
2. The reason behind is that we cannot train the LLM each time for the small tiny tasks, so we just give access to RAG Vector Database which stores data in embedded form.

**Embedding**

AI converts our prompt into embedding form and then goes to the Embedded database to search our required information. For understanding we divide the process of Embedding in for steps. Embedding is a model trained on deeplearning
1. We collect the data.
2. We categorrise/split the data on the bases of categories i.e. Text, Image, Video, Audio etc.
3. We Embed the data in which our data is converted into numbers which machine understands and assigns the suitable numbers on the basis of category and then when we give the prompt the machine converts our prompt in numbers and then matches the numbers of prompt keywords with the stored/embedded data through searching and gives the results. So, in step 3. the categorise data is embedded in numbers with the help of pre-trained Models that embed our data into numbers. So, the most important thing is to select relevant embedding model related to your domain.
4. In step 4: the data is stored in Vector Database after embedding  that is used later on when required.

**Vector Search Formula: Concise Formula**

Vector search formula also known as Cousine Formual; it telss angle between two vectors. If the angle is close/nearer that means the relationship or relativity is strong and vice versa. For this purpose https://projector.tensorflow.org/ can be seen for the better understandig about vector search.


In [1]:
%pip install -qU langchain-pinecone langchain-google-genai
#-qu means to install latest packages by sending not much messages
# secondly we used the rappers of pipecone and google genai API's so that we can call them consistently.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00


Initializing Pinecone by Adding Pinecone API Key in Colab. In case of local development it is not required. But we are doing on Colab so that not only we can share but also we have all the installation already setup in colab environment. Here ServerlessSpec is a library provided by Pinecone, it makes the database serverless. Pinecone is using cloud aws/azure and when we make a database it asks where to save you database by this library (ServerlessSpec)

In [2]:
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
pinecone_api_key = userdata.get('PINECONE_API_KEY')
pc = Pinecone(api_key=pinecone_api_key) #here we initialize the object of pinecone pc.

Creating Index in Pinecone Vector Database

Creating Index in Pinecone Vector Database
Dimensions = There are three types of dimensions, 384, 512 and 768 and hese dimensions depeend on accuracy. For example if we ask a question from the LLM it will go in 768 dimensions, 384 or 512 dimension and will match the answer and will give the answer. However, high dimension model will be big one and will give more accuracy, however, we will use more GPUs of our system. Because it have 768 norms/neural networks/contacts/direction, reachability in our provided data to search the answers of our prompt.Fewer than this 512 will give less accuracy results and then 384 will be general nature response from the LLM and it will not go in detail.

In [4]:
index_name = "project-2-rag"
pc.create_index(
            name=index_name,
            dimension=768,  # Dimensions of the embeddings
            metric="cosine",  # Metric to use for distance calculations
            spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )
index = pc.Index(index_name)

Embedding: Here we set the embedding model and make instance of our Embedding class.

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY') #We get api from google colab through userdata and give it to the system so that it can get whenever required through system environment.
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001") # we can search Chtgpt to find related model of embedding

From the above class GoogleGenerativeAIEmbeddings we have initialized the class and have created an object embeddings and that embeddings object may have many functions. Below function "embed_query" is the function of embeddings. By this function we have embedded the words "Hello World" and have presented the embedded data in digits.

In [6]:
vector = embeddings.embed_query("hello world")

this will print the vectors of above word "Hello World"

In [7]:
vector[:4]

[0.04909781739115715,
 -0.044328317046165466,
 -0.025365281850099564,
 -0.030721040442585945]

We already make index in previous cells, In this process we are rapping index in another layer PineConeVectorStore that is a class of pinecone and whenever we will need we will used this one.

In [8]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In this process we made 10 documents and put them in an array.
UUIDs used to give IDS to our documents. UUID creates random string each time it is called.

In [23]:
#Data Save
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)
document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)
document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)
document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)
document_5 = Document(
    page_content="Wow! That was an amazing movie according to the requirements of the weather. I can't wait to see it again.",
    metadata={"source": "tweet"},
)
documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
]

In [10]:
len(documents)

5

Now we are giving unique ID to our all documents and store them in vector store by function add_documents.
This method (add documents) uses documents and ids and then it creates embeddings.

In [24]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['acac5f33-d419-4d00-a19b-499ce35140e9',
 'd0619e43-59c2-4fe4-abd5-333c5376c8e5',
 'ae4c782e-1d23-4fe0-b78e-0cc510b84a12',
 '6149ecd7-61af-4a65-8f21-afee0fe38928',
 '95af6f26-92dc-4956-b036-946c28e7d42a']

Here we asked to provide two documents which should have nearby results of the prompt "Langchain provide.....easy"

In [25]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2, #Specifies that you want to retrieve the top2 most similar documents.
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")
#This line prints the content (page_content) of each result, along with its associated metadata.
#(metadata) which in this case includes the source.

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


Here we will get the score of our similarity index.

In [28]:
results = vector_store.similarity_search_with_score(
    "What about weather?", k=2, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.622348] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.622348] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [31]:
def answer_to_user(query: str):
        #Vector Search
    vector_results = vector_store.similarity_search(query, k=2)
    print(len(vector_results))
    final_answer = llm.invoke(f"ANSWER THIS USER QUERY: {query}, Here are some reference to answer {vector_results}")
    return final_answer


In [33]:
answer = answer_to_user("What is discussed about weather?")
print(answer.content)

2
The provided text mentions weather in two different ways.  One mentions a movie being "amazing according to the requirements of the weather," which is vague and doesn't specify what weather conditions were involved. The other provides a specific weather forecast: cloudy and overcast with a high of 62 degrees.
